In [5]:
## import models 
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np


In [6]:
# Loading all the pickle files
model = load_model('customer_churn_model.h5')

## Loading the scaler and encoders
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

with open('onehot_encoder_geography.pkl', 'rb') as f:
    onehot_encoder_geog = pickle.load(f)

with open('label_encoder_gender.pkl', 'rb') as f:
    label_encoder_gender = pickle.load(f)


In [7]:
# Sample data for single customer churn prediction
sample_data = {
    'CreditScore': 619,
    'Geography': 'France',
    'Gender': 'Female',
    'Age': 42,
    'Tenure': 2,
    'Balance': 83807.86,
    'NumOfProducts': 1,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 101348.88
}


In [10]:
#Applying One hot encoding in geography
coded = onehot_encoder_geog.transform([[sample_data['Geography']]]).toarray()
geo_onhot_df = pd.DataFrame(coded, columns=onehot_encoder_geog.get_feature_names_out(['Geography']))
geo_onhot_df


c:\Users\mdama\OneDrive\Desktop\customerChunPrediction_ANN\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [13]:
#Combine one hot encoded geography column with sample data
sample_df = pd.DataFrame([sample_data])
sample_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,83807.86,1,1,1,101348.88


In [14]:
#Encoding gender
sample_df['Gender'] = label_encoder_gender.transform(sample_df['Gender'])
sample_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,0,42,2,83807.86,1,1,1,101348.88


In [15]:
## Concatenate the one hot encoded geography columns with the sample data
final_sample_df = pd.concat([sample_df.drop(['Geography'], axis=1), geo_onhot_df], axis=1)
final_sample_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,83807.86,1,1,1,101348.88,1.0,0.0,0.0


In [16]:
#Scalling the data
scaled_sample = scaler.transform(final_sample_df)
scaled_sample

array([[-0.33880827, -1.09499335,  0.29493847, -1.04241787,  0.12337621,
        -0.91668767,  0.64920267,  0.97481699,  0.01595384,  1.00150113,
        -0.57946723, -0.57638802]])

In [17]:
##Predicting the churn
prediction = model.predict(scaled_sample)
prediction_label = (prediction > 0.5).astype(int)   
print(f"Churn Probability: {prediction[0][0]:.4f}")

1/1 [==============================] - 0s 491ms/step
Churn Probability: 0.3589


0
